# Day 6: Probably a Fire Hazard

[*Advent of Code 2015 day 6*](https://adventofcode.com/2015/day/6) and [*solution megathread*](https://redd.it/3vmltn)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2015/06/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2015%2F06%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

Writing 'downloaded' (dict) to file 'downloaded'.


In [2]:
HTML(downloaded['part1'])

In [3]:
testdata = ['toggle 0,0 through 999,999',  # would turn on (or leave on) every light.
            'turn on 0,0 through 999,0',  # would toggle the first line of 1000 lights, turning off the ones that were on, and turning on the ones that were off.
            'toggle 0,0 through 500,500']  # would turn off (or leave off) the middle four lights.

inputdata = downloaded['input'].splitlines()

In [4]:
inputdata[1:5]

['turn off 341,304 through 638,850',
 'turn off 199,133 through 461,193',
 'toggle 322,558 through 977,958',
 'toggle 537,781 through 687,941']

In [5]:
from enum import Enum

class LightOperation(Enum):
    turn_on = 'turn on'
    turn_off = 'turn off'
    toggle = 'toggle'

    def __str__(self):
        return self.value

class LightInstruction:
    x1 = y1 = x2 = y2 = 0
    operation = LightOperation('toggle')

    def __init__(self, instruction_string: str):
        tokens = instruction_string.split(' ')
        self.x1, self.y1 = map(int, tokens[-3].split(','))
        self.x2, self.y2 = map(int, tokens[-1].split(','))
        self.operation = LightOperation(' '.join(tokens[:-3]))

    def __str__(self):
        return(f'{str(self.operation)} {str(self.x1)},{str(self.y1)} through {str(self.x2)},{str(self.y2)}')
    
    def __repr__(self):
        return self.__str__()

In [6]:
from itertools import product

class MillionLights:
    matrix = [[]]

    def __init__(self):
        self.matrix = [[False for _ in range(1000)] for _ in range(1000)]

    def execute_operation(self, x: int, y: int, operation: LightOperation):
            if operation == LightOperation.turn_off:
                self.matrix[y][x] = False
            elif operation == LightOperation.turn_on:
                self.matrix[y][x] = True
            else:  # operation == LightOperation.toggle:
                self.matrix[y][x] ^= True

    def execute_instruction(self, instruction: LightInstruction) -> int:
        for x, y in product(range(instruction.x1, instruction.x2 + 1),
                            range(instruction.y1, instruction.y2 + 1)):
            self.execute_operation(x, y, instruction.operation)
        return self.count_lit()

    def execute_instructions(self, instructions) -> int:
        # for x, y in product(range(5), range(5)):
        for x, y in product(range(1000), range(1000)):
            applicable = [instruction for instruction in instructions
                            if instruction.x1 <= x and instruction.y1 <= y
                            and instruction.x2 >= x and instruction.y2 >= y]
            try:
                relevant_i = max([i for i, instruction in enumerate(applicable) if instruction.operation == LightOperation.turn_on] +
                                [i for i, instruction in enumerate(applicable) if instruction.operation == LightOperation.turn_off]) 
                self.execute_operation(x, y, applicable[relevant_i].operation)
            except ValueError:
                relevant_i = -1
            
            toggles = len(applicable) - relevant_i + 1
            if toggles % 2 == 1:
                self.execute_operation(x, y, LightOperation.toggle)

        return self.count_lit()

    def count_lit(self) -> int:
        return sum(sum(row) for row in self.matrix)

In [7]:
for instruction_string in testdata:
    # print(instruction_string)
    ml = MillionLights()
    print(ml.execute_instruction(LightInstruction(instruction_string)))

1000000
1000
251001


In [8]:
ml = MillionLights()
print(ml.execute_instructions(list(map(LightInstruction, testdata))))

748999


In [9]:
def my_part1_solution(data):
    ml = MillionLights()
    return(ml.execute_instructions(list(map(LightInstruction, data))))

In [10]:
import timeit
starttime = timeit.default_timer()
print(my_part1_solution(inputdata))
print("The time difference is :", timeit.default_timer() - starttime)

400410
The time difference is : 109.15800300001865


In [11]:
import timeit
starttime = timeit.default_timer()
ml = MillionLights()
for instruction in inputdata:
    ml.execute_instruction(LightInstruction(instruction))
print(ml.count_lit())
print("The time difference is :", timeit.default_timer() - starttime)

400410
The time difference is : 33.49445120000746


In [12]:
HTML(downloaded['part1_footer'])

In [13]:
HTML(downloaded['part2'])

In [14]:
class LightOperation2(Enum):
    turn_on = 'turn on'
    turn_off = 'turn off'
    toggle = 'toggle'

    def __str__(self):
        return self.value
    
    def op(self):
        if self == LightOperation2.turn_on:
            return lambda l: l + 1
        elif self == LightOperation2.turn_off:
            return lambda l: l - 1 if l > 0 else 0
        else:  # self == LightOperation2.toggle
            return lambda l: l + 2

class LightInstruction2:
    x1 = y1 = x2 = y2 = 0
    operation = LightOperation('toggle')

    def __init__(self, instruction_string: str):
        tokens = instruction_string.split(' ')
        self.x1, self.y1 = map(int, tokens[-3].split(','))
        self.x2, self.y2 = map(int, tokens[-1].split(','))
        self.operation = LightOperation2(' '.join(tokens[:-3]))

    def __str__(self):
        return(f'{str(self.operation)} {str(self.x1)},{str(self.y1)} through {str(self.x2)},{str(self.y2)}')
    
    def __repr__(self):
        return self.__str__()

In [15]:
class MillionLights2:
    matrix = [[]]

    def __init__(self):
        self.matrix = [[0 for _ in range(1000)] for _ in range(1000)]

    def execute_operation(self, x: int, y: int, operation: LightOperation2):
            self.matrix[y][x] = operation.op()(self.matrix[y][x])

    def execute_instruction(self, instruction: LightInstruction2) -> int:
        for x, y in product(range(instruction.x1, instruction.x2 + 1),
                            range(instruction.y1, instruction.y2 + 1)):
            self.execute_operation(x, y, instruction.operation)
        return self.count_lit()

    def execute_instructions(self, instructions) -> int:
        # for x, y in product(range(5), range(5)):
        for x, y in product(range(1000), range(1000)):
            applicable = [instruction for instruction in instructions
                            if instruction.x1 <= x and instruction.y1 <= y
                            and instruction.x2 >= x and instruction.y2 >= y]
            for instruction in applicable:
                self.execute_operation(x, y, instruction.operation)
        return self.count_lit()

    def count_lit(self) -> int:
        return sum(sum(row) for row in self.matrix)

In [16]:
starttime = timeit.default_timer()
ml = MillionLights2()
for instruction in inputdata:
    ml.execute_instruction(LightInstruction2(instruction))
print(ml.count_lit())
print("The time difference is :", timeit.default_timer() - starttime)

15343601
The time difference is : 43.186366100009764


In [17]:
HTML(downloaded['part2_footer'])